### Set path to original pyNeuroChem. Please change to your own path

In [1]:
import os
os.environ["ANI_HOME"] = "/home/olexandr/ani/NeuroChem-ANI1"

import sys
sys.path.append('/home/olexandr/notebooks/ASE_ANI')
from ani import ANI

In [2]:
import numpy as np
import time
# ASE
import  ase
from ase.io import read, write
from ase.optimize import BFGS, LBFGS 
from ase.vibrations import Vibrations

#figure plotting
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Read geometry from xyz file

In [3]:
geometry = read('data/water.xyz')

Setup ANI and calculate single point energy

In [4]:
geometry.set_calculator(ANI())
e = geometry.get_potential_energy()
print('Tital energy', e, 'eV')

Tital energy -2078.62183806 eV


In [5]:
geometry.get_forces()

array([[ 0.10618784, -0.28556702,  0.00676665],
       [-0.1251156 ,  0.30097666, -0.00710004],
       [ 0.01892776, -0.01540952,  0.00033338]], dtype=float32)

Geometry optimization with BFGS

In [6]:
start_time = time.time()
dyn = LBFGS(geometry)
dyn.run(fmax=0.001)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

LBFGS:   0  17:58:50    -2078.621838       0.3260
LBFGS:   1  17:58:50    -2078.622792       0.1269
LBFGS:   2  17:58:50    -2078.622943       0.0280
LBFGS:   3  17:58:50    -2078.622973       0.0253
LBFGS:   4  17:58:50    -2078.623015       0.0173
LBFGS:   5  17:58:50    -2078.623023       0.0044
LBFGS:   6  17:58:50    -2078.623024       0.0005
[ANI Total time: 0.03528308868408203 seconds]


In [7]:
e = geometry.get_potential_energy()
print('Tital energy', e, 'eV')

Tital energy -2078.62302368 eV


In [8]:
geometry.get_forces()

array([[ -3.16782207e-05,  -5.18357090e-04,   1.28185929e-05],
       [ -2.76740957e-05,   4.01983969e-04,  -9.82341680e-06],
       [  5.90988930e-05,   1.16423806e-04,  -3.00844113e-06]], dtype=float32)

Vibrations and thermochemistry with numeric differentiation

In [9]:
vib = Vibrations(geometry, nfree=2) 
vib.run()

Writing vib.eq.pckl
Writing vib.0x-.pckl
Writing vib.0x+.pckl
Writing vib.0y-.pckl
Writing vib.0y+.pckl
Writing vib.0z-.pckl
Writing vib.0z+.pckl
Writing vib.1x-.pckl
Writing vib.1x+.pckl
Writing vib.1y-.pckl
Writing vib.1y+.pckl
Writing vib.1z-.pckl
Writing vib.1z+.pckl
Writing vib.2x-.pckl
Writing vib.2x+.pckl
Writing vib.2y-.pckl
Writing vib.2y+.pckl
Writing vib.2z-.pckl
Writing vib.2z+.pckl


In [10]:
vib.summary()

---------------------
  #    meV     cm^-1
---------------------
  0    1.2i      9.4i
  1    0.2i      1.3i
  2    0.1i      0.6i
  3    2.7      21.6 
  4    3.0      24.0 
  5    3.2      25.8 
  6  173.7    1401.4 
  7  438.2    3534.5 
  8  450.3    3631.8 
---------------------
Zero-point energy: 0.536 eV


In [11]:
vib.get_zero_point_energy()

0.53555131135295064

In [12]:
vib.get_frequencies()

array([    0.00000000+9.3936345j ,     0.00000000+1.34240613j,
           0.00000000+0.57014453j,    21.55650642+0.j        ,
          23.98202197+0.j        ,    25.82109748+0.j        ,
        1401.35703823+0.j        ,  3534.53983715+0.j        ,
        3631.76883583+0.j        ])

In [13]:
# remove mode pckl files
vib.clean()

19